In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

In [2]:
def diode_equation(VT, Id, n):
    e = 1.602e-19
    kb = 1.38e-23
    return Id*(np.exp(VT*e/kb/n)-1)

def diode_equation_error(VT, Id, n, VT_err, Id_err, n_err):
    # varaince calculation
    e = 1.602e-19
    kb = 1.38e-23
    return (Id*e/kb/n)**2*np.exp(2*e*VT/kb/n)*VT_err + (np.exp(e*VT/kb/n)-1)**2*Id_err + (Id*e*VT/kb*n**(-2))**2*np.exp(2*e*VT/kb/n)*n_err

def chi_squared(exp, th, err):
    if len(exp) != len(th):
        raise ValueError('dimensions do not match')
        
    return {'theoretical_chi_2':len(exp), 'chi2_deviation':np.sqrt(2*len(exp)), 'calculated_chi2':np.sum((exp-th)**2/err)}

In [3]:
df = pd.read_excel('pn_junction_30.2.xlsx')
df

,Applied Voltage,Resistor Voltage,Temperature,Voltage Uncertainty,Temperature Uncertainty
0,0.336,0.173,30.2,0.001,0.1
1,0.554,0.353,30.2,0.001,0.1
2,0.801,0.577,30.2,0.001,0.1
3,1.119,0.872,30.2,0.001,0.1
4,1.512,1.248,30.2,0.001,0.1
5,1.795,1.520,30.2,0.001,0.1


In [4]:
resistance = 10e6
data = df.values.T
V_diode = data[0] - data[1]
I_meas = data[1] / resistance
temps = data[2]+273.15
ins = V_diode/temps
outs = I_meas
ins_unc = ((data[3]/V_diode)**2 + (data[4]/temps)**2)*(V_diode/temps)**2

In [5]:
init = [10e-9,2]
params, cov = curve_fit(diode_equation, ins, outs, p0=init)

Id = params[0]
Id_error = cov[0,0]

n = params[1]
n_error = cov[1,1]

err = diode_equation_error(ins, Id, n, ins_unc, Id_error, n_error)

print('Saturation current {} amps with error {} amps \nIdeality factor {} with error {}'.format(Id,Id_error,n,n_error))

Saturation current 7.667002227721072e-10 amps with error 4.017616671102083e-21 amps 
Ideality factor 1.986181364174956 with error 0.0010411433113352746


In [7]:
init = [10e-9,2]
params, cov = curve_fit(diode_equation, ins, outs, p0=init, sigma=1/err)

Id = params[0]
Id_error = cov[0,0]

n = params[1]
n_error = cov[1,1]

err = diode_equation_error(ins, Id, n, ins_unc, Id_error, n_error)

print('Saturation current {} amps with error {} amps \nIdeality factor {} with error {}'.format(Id,Id_error,n,n_error))

Saturation current 8.818384673015002e-10 amps with error 1.470833737602632e-20 amps 
Ideality factor 2.0407420206245934 with error 0.0029891573929707064
